### Importing the neccessary packages

In [1]:
import pandas as pd
from pandasql import sqldf
from sqlalchemy import create_engine
import mysql.connector
import pymysql

### MySQL database connection
I downloaded the files inform csv then load them to MyQSL server to perform analysis locally

In [2]:
engine = create_engine(f'mysql+pymysql://{"root"}:{""}@{"localhost"}/{"forestquery_db"}', pool_recycle=3306)
dbConnection = engine.connect()
print(dbConnection)

In [3]:
forestation = pd.read_sql("forestation", dbConnection)

### Importing the downloaded csv files from Udacity to be used locally
They are regions.csv,forest_are.csv,land_area.csv

In [4]:
 regions = pd.read_csv('data/regions.csv')

In [5]:
 forest_area = pd.read_csv('data/forest_area.csv')

In [8]:
 land_area = pd.read_csv('data/land_area.csv')

In [14]:
 forestation = pd.read_csv('data/forestation.csv')

In [15]:
forestation.head()

,country_code,country_name,year,forest_area_sqkm,total_area_sqkm,region,income_group,forest_in_percent
0,ABW,Aruba,2016,4.20000,1.800050e+02,Latin America & Caribbean,High income,0.023333
1,AFG,Afghanistan,2016,13500.00000,6.528600e+05,South Asia,Low income,0.020678
2,AGO,Angola,2016,577311.99220,1.246700e+06,Sub-Saharan Africa,Lower middle income,0.463072
3,ALB,Albania,2016,7705.39978,2.740000e+04,Europe & Central Asia,Upper middle income,0.281219
4,AND,Andorra,2016,160.00000,4.700073e+02,Europe & Central Asia,High income,0.340420


In [6]:
forestation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5886 entries, 0 to 5885
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   country_code      5886 non-null   object 
 1   country_name      5886 non-null   object 
 2   year              5886 non-null   int64  
 3   forest_area_sqkm  5570 non-null   float64
 4   total_area_sqkm   5808 non-null   float64
 5   region            5886 non-null   object 
 6   income_group      5859 non-null   object 
 7   forest_perc       5534 non-null   float64
dtypes: float64(3), int64(1), object(4)
memory usage: 368.0+ KB


### Creating tables in the MySQL database name forestquery_db

In [16]:
forest_area.to_sql("forest_area", engine, index=None)

In [ ]:
land_area.to_sql("land_area", engine, index=None)

In [ ]:
regions.to_sql("regions", engine, index=None)

### Analysis starts here

In [11]:
# limiting to 5 rows for viewing sake
#  ROUND(CAST(SUM(forest_area_sqkm)/SUM(total_area_sqkm)*100 AS NUMERIC), 2) 
# round(your_expression::numeric,2)
query = """

SELECT country_code, country_name, year, ROUND(total_area_sq_mi*2.59, 2) as total_area_sqkm
FROM land_area
LIMIT 5

"""
total_area_sqkm = sqldf(query)

In [12]:
total_area_sqkm

,country_code,country_name,year,total_area_sqkm
0,ABW,Aruba,2016,180.01
1,AFG,Afghanistan,2016,652860.01
2,AGO,Angola,2016,1246700.00
3,ALB,Albania,2016,27400.00
4,AND,Andorra,2016,470.01


#### Steps to Complete
Create a View called “forestation” by joining all three tables - forest_area, land_area and regions in the workspace.
The forest_area and land_area tables join on both country_code AND year.
The regions table joins these based on only country_code.
In the ‘forestation’ View, include the following:

All of the columns of the origin tables
A new column that provides the percent of the land area that is designated as forest.
Keep in mind that the column forest_area_sqkm in the forest_area table and the land_area_sqmi in the land_area table are in different units (square kilometers and square miles, respectively), so an adjustment will need to be made in the calculation you write (1 sq mi = 2.59 sq km).

<img src='data/forestation-view.png' alt='drawing' width='400'/>

#### Creating forestation view

In [12]:
query = """

DROP table IF EXISTS forestation;

"""
forestation = sqldf(query)


In [13]:
# query = """

# CREATE view forestation
# AS(SELECT f.country_code,
#           f.country_name,
#           f.year,
#           f.forest_area_sqkm,
#           l.total_area_sq_mi * 2.59 AS total_area_sqkm,
#           r.region,
#           r.income_group,
#           forest_area_sqkm / ( l.total_area_sq_mi * 2.59 ) AS forest_perc
#    FROM   forest_area f
#           JOIN land_area l
#             ON f.country_code = l.country_code
#                AND f.year = l.year
#           JOIN regions r
#             ON r.country_code = l.country_code); 

# """
forestation = sqldf(query)

#### GLOBAL SITUATION
Instructions on how to answer the questions

Answering these questions will help you add information into the template.
Use these questions as guides to write SQL queries.
Use the output from the query to answer these questions.

a. What was the total forest area (in sq km) of the world in 1990? 
Please keep in mind that you can use the country record denoted as “World" in the region table.

b. What was the total forest area (in sq km) of the world in 2016? 
Please keep in mind that you can use the country record in the table is denoted as “World.”

c. What was the change (in sq km) in the forest area of the world from 1990 to 2016?

d. What was the percent change in forest area of the world between 1990 and 2016?

e. If you compare the amount of forest area lost between 1990 and 2016, to which country's total area in 2016 is it closest to?

In [6]:
# Query results for country name World
# limiting to 5 rows for viewing sake
query = """
SELECT * 
FROM forestation
WHERE country_name = 'World'
LIMIT 5
"""
country_name_world = sqldf(query)

In [20]:
country_name_world

,country_code,country_name,year,forest_area_sqkm,total_area_sqkm,region,income_group,forest_in_percent
0,WLD,World,2016,39958245.9,1.273546e+08,World,None,0.313756
1,WLD,World,2015,39991324.6,1.273444e+08,World,None,0.314041
2,WLD,World,2014,40024403.3,1.273450e+08,World,None,0.314299
3,WLD,World,2013,40057482.0,1.273463e+08,World,None,0.314555
4,WLD,World,2012,40090560.5,1.273474e+08,World,None,0.314813


In [15]:
# What was the total forest area (in sq km) of the world in 1990? 
# Please keep in mind that you can use the country record denoted as “World" in the region table.
query = """

SELECT year, country_name, forest_area_sqkm 
FROM forestation 
WHERE country_name = 'World' AND year = 1990
    
"""
forest_area_in_1990 = sqldf(query)

In [16]:
forest_area_in_1990 

,year,country_name,forest_area_sqkm
0,1990,World,41282694.9


In [23]:
# What was the total forest area (in sq km) of the world in 2016? 
# Please keep in mind that you can use the country record in the table is denoted as “World.”

query = """

SELECT year, country_name, forest_area_sqkm 
FROM forestation 
WHERE country_name = 'World' AND year = 2016
    
"""
forest_area_in_2016 = sqldf(query)

In [24]:
forest_area_in_2016

,year,country_name,forest_area_sqkm
0,2016,World,39958245.9


In [25]:
# Sub-query result to be used to answer the following question

query = """

SELECT a.forest_area_sqkm in_1990, b.forest_area_sqkm in_2016
FROM forestation a,
     forestation b
WHERE a.year = 1990
    AND b.year = 2016
    AND a.country_name = 'World'
    AND b.country_name = 'World'
 
"""
sqldf(query)

,in_1990,in_2016
0,41282694.9,39958245.9


In [19]:
# What was the change (in sq km) in the forest area of the world from 1990 to 2016?

query = """
WITH forest_area_in_1990_to_2016
     AS (SELECT a.forest_area_sqkm AS in_1990,
                b.forest_area_sqkm AS in_2016
         FROM   forestation a,
                forestation b
         WHERE  a.year = 1990
                AND b.year = 2016
                AND a.country_name = 'World'
                AND b.country_name = 'World')
SELECT in_2016 - in_1990 AS forest_area_change_btw_1990_to_2016
FROM   forest_area_in_1990_to_2016;   
"""
forest_area_change_btw_1990_to_2016 = sqldf(query)

In [20]:
forest_area_change_btw_1990_to_2016

,forest_area_change_btw_1990_to_2016
0,-1324449.0


In [21]:
# What was the percentage loss (in sq km) in the forest area of the world from 1990 to 2016?

query = """
WITH forest_loss_perc_1990_2016 AS 
(SELECT a.forest_area_sqkm AS in_1990, b.forest_area_sqkm AS in_2016
FROM forestation a, forestation b
WHERE a.year = 1990
    AND b.year = 2016
    AND a.country_name = 'World'
    AND b.country_name = 'World')
SELECT ROUND((in_2016 - in_1990)*100/in_1990, 2) AS forest_loss_perc_btw_1990_2016
FROM forest_loss_perc_1990_2016;
"""
forest_loss_perc_btw_1990_2016 = sqldf(query)

In [22]:
forest_loss_perc_btw_1990_2016

,forest_loss_perc_btw_1990_2016
0,-3.21


In [28]:
query = f"""
SELECT country_name, total_area_sqkm 
FROM forestation 
WHERE year = 2016 AND total_area_sqkm <= (
WITH forest_loss_perc_1990_2016 AS (
SELECT a.forest_area_sqkm in_1990, b.forest_area_sqkm in_2016
FROM forestation a, forestation b
WHERE a.year = 1990
    AND b.year = 2016
    AND a.country_name = 'World'
    AND b.country_name = 'World')
SELECT ABS(in_2016 - in_1990) country_close_to_forest_area_lost
FROM forest_loss_perc_1990_2016 )
ORDER BY 2 DESC
LIMIT 1 ;

"""
country_close_to_forest_area_lost = sqldf(query)

In [30]:
country_close_to_forest_area_lost

,country_name,total_area_sqkm
0,World,1.273546e+08


In [31]:
# If you compare the amount of forest area lost between 1990 and 2016, 
# to which country's total area in 2016 is it closest to?

query = f"""

SELECT country_name, total_area_sqkm 
FROM forestation 
WHERE year = 2016 AND total_area_sqkm <= "forest_loss_perc_btw_1990_2016"
ORDER BY 2 DESC
LIMIT 1 ;
"""
country_close_to_forest_area_lost = sqldf(query)


In [32]:
country_close_to_forest_area_lost

,country_name,total_area_sqkm
0,World,1.273546e+08


#### REGIONAL OUTLOOK
Instructions on how to answer the questions

Answering these questions will help you add information into the template.
Use these questions as guides to write SQL queries.
Use the output from the query to answer these questions.

Create a table that shows the Regions and their percent forest area (sum of forest area divided by sum of land area) in 1990 and 2016. (Note that 1 sq mi = 2.59 sq km).
Based on the table you created, ....

a. What was the percent forest of the entire world in 2016? Which region had the HIGHEST percent forest in 2016, and which had the LOWEST, to 2 decimal places?

b. What was the percent forest of the entire world in 1990? Which region had the HIGHEST percent forest in 1990, and which had the LOWEST, to 2 decimal places?

c. Based on the table you created, which regions of the world DECREASED in forest area from 1990 to 2016?

In [32]:
# What was the percent forest of the entire world in 2016? 
query = """
SELECT * 
FROM forestation
WHERE country_name = 'World' AND year = 2016;
"""
forest_area_2016 = sqldf(query)


In [33]:
forest_area_2016

,country_code,country_name,year,forest_area_sqkm,total_area_sqkm,region,income_group,forest_in_percent
0,WLD,World,2016,39958245.9,1.273546e+08,World,None,0.313756


In [34]:
# Which regions had the HIGHEST percent forest in 2016, 

query = """
SELECT  region, SUM(forest_area_sqkm) AS forest_area_sum, 
        SUM(total_area_sqkm) AS total_area_sum, 
        ROUND(CAST(SUM(forest_area_sqkm)/SUM(total_area_sqkm)*100 AS NUMERIC), 2) AS highest_percent_forest_2016
FROM forestation
WHERE year = 2016
GROUP BY 1
ORDER BY 4 DESC
LIMIT 1
"""
highest_percent_forest_2016 = sqldf(query)

In [35]:
highest_percent_forest_2016

,region,forest_area_sum,total_area_sum,highest_percent_forest_2016
0,Latin America & Caribbean,9.250586e+06,2.003936e+07,46.16


In [36]:
# Which regions had the LOWEST percent forest in 2016, 

query = """
SELECT region,  SUM(forest_area_sqkm) AS forest_area_sum, 
        SUM(total_area_sqkm) AS total_area_sum, 
        ROUND(CAST(SUM(forest_area_sqkm)/SUM(total_area_sqkm)*100 AS NUMERIC), 2) AS highest_percent_forest_2016
FROM forestation
WHERE year = 2016
GROUP BY 1
ORDER BY 4 ASC
LIMIT 1
"""
lowest_percent_forest_2016 = sqldf(query)

In [37]:
lowest_percent_forest_2016

,region,forest_area_sum,total_area_sum,highest_percent_forest_2016
0,Middle East & North Africa,232131.00401,1.122347e+07,2.07


In [38]:
# What was the percent forest of the entire world in 1990? 
query = """
SELECT * 
FROM forestation
WHERE country_name = 'World' AND year = 1990;
"""
forest_area_1990 = sqldf(query)


In [39]:
forest_area_1990

,country_code,country_name,year,forest_area_sqkm,total_area_sqkm,region,income_group,forest_in_percent
0,WLD,World,1990,41282694.9,1.273285e+08,World,None,0.324222


In [40]:
# Which regions had the HIGHEST percent forest in 1990, 

query = """
SELECT  region, SUM(forest_area_sqkm) forest_area_sum, 
        SUM(total_area_sqkm) total_area_sum, 
        ROUND(CAST(SUM(forest_area_sqkm)/SUM(total_area_sqkm)*100 AS NUMERIC), 2) highest_percent_forest_1990 
FROM forestation
WHERE year = 1990
GROUP BY 1
ORDER BY 4 DESC
LIMIT 1
"""
highest_percent_forest_1990 = sqldf(query)


In [41]:
highest_percent_forest_1990

,region,forest_area_sum,total_area_sum,highest_percent_forest_1990
0,Latin America & Caribbean,1.024234e+07,2.007122e+07,51.03


In [38]:
# Which regions had the LOWEST percent forest in 1990, 

query = """
SELECT  region, SUM(forest_area_sqkm) forest_area_sum, 
        SUM(total_area_sqkm) total_area_sum, 
        ROUND(CAST(SUM(forest_area_sqkm)/SUM(total_area_sqkm)*100 AS NUMERIC), 2) highest_percent_forest_1990 
FROM forestation
WHERE year = 1990
GROUP BY 1
ORDER BY 4 ASC
LIMIT 1
"""
lowest_percent_forest_1990 = sqldf(query)

In [41]:
lowest_percent_forest_1990

,region,forest_area_sum,total_area_sum,highest_percent_forest_1990
0,Middle East & North Africa,199292.595699,1.122623e+07,1.78


In [52]:
query = f"""

SELECT region,
                    Sum(forest_area_sqkm) forest_area_sum,
                    Sum(total_area_sqkm)  total_area_sum,
                    Round(Cast(Sum(forest_area_sqkm) / Sum(total_area_sqkm) *100 AS NUMERIC), 2) AS forest_area_decrease_btw_1990_2016                          
             FROM   forestation
             WHERE  year = 2016
             GROUP  BY 1
             
UNION

SELECT region,
               Sum(forest_area_sqkm) AS forest_area_sum,
               Sum(total_area_sqkm)  total_area_sum,
               Round(Cast(Sum(forest_area_sqkm) / Sum(total_area_sqkm) * 100 AS NUMERIC ), 2) AS forest_area_decrease_btw_1990_2016                   
        FROM   forestation
        WHERE  year = 1990
        GROUP  BY 1
        ORDER  BY 4 DESC

"""
union_percent_forest_1990_2016 = sqldf(query)

In [53]:
union_percent_forest_1990_2016

,region,forest_area_sum,total_area_sum,forest_area_decrease_btw_1990_2016
0,Latin America & Caribbean,1.024234e+07,2.007122e+07,51.03
1,Latin America & Caribbean,9.250586e+06,2.003936e+07,46.16
2,Europe & Central Asia,1.043861e+07,2.744011e+07,38.04
3,Europe & Central Asia,1.019985e+07,2.735722e+07,37.28
4,North America,6.573934e+06,1.824098e+07,36.04
5,North America,6.507240e+06,1.825252e+07,35.65
6,World,4.128269e+07,1.273285e+08,32.42
7,World,3.995825e+07,1.273546e+08,31.38
8,Sub-Saharan Africa,6.515615e+06,2.124139e+07,30.67
9,Sub-Saharan Africa,6.115291e+06,2.124236e+07,28.79


In [49]:
percent_forest_1990

,region,forest_area_sum,total_area_sum,forest_area_decrease_btw_1990_2016
0,Latin America & Caribbean,9.250586e+06,2.003936e+07,46.16
1,Europe & Central Asia,1.043861e+07,2.744011e+07,38.04
2,North America,6.573934e+06,1.824098e+07,36.04
3,World,3.995825e+07,1.273546e+08,31.38
4,Sub-Saharan Africa,6.115291e+06,2.124236e+07,28.79
5,East Asia & Pacific,6.421326e+06,2.436134e+07,26.36
6,South Asia,8.353105e+05,4.771604e+06,17.51
7,Middle East & North Africa,2.321310e+05,1.122347e+07,2.07


In [54]:
# Based on the table you created, which regions of the world DECREASED in forest area from 1990 to 2016?
query = f"""
SELECT a.region,
       a.forest_area_decrease_btw_1990_2016 AS forest_perc_1990,
       b.forest_area_decrease_btw_1990_2016 AS forest_perc_2016,
       b.forest_area_decrease_btw_1990_2016 - a.forest_area_decrease_btw_1990_2016 AS forest_perc_decrease
FROM   (SELECT region,
               Sum(forest_area_sqkm) AS forest_area_sum,
               Sum(total_area_sqkm)  total_area_sum,
               Round(Cast(Sum(forest_area_sqkm) / Sum(total_area_sqkm) * 100 AS NUMERIC ), 2) AS forest_area_decrease_btw_1990_2016                   
        FROM   forestation
        WHERE  year = 1990
        GROUP  BY 1
        ORDER  BY 4 DESC) a
       JOIN (SELECT region,
                    Sum(forest_area_sqkm) forest_area_sum,
                    Sum(total_area_sqkm)  total_area_sum,
                    Round(Cast(Sum(forest_area_sqkm) / Sum(total_area_sqkm) *100 AS NUMERIC), 2) AS forest_area_decrease_btw_1990_2016                          
             FROM   forestation
             WHERE  year = 2016
             GROUP  BY 1
             ORDER  BY 4 DESC) b
         ON a.region = b.region
ORDER  BY 4; 

"""
forest_area_decrease_btw_1990_2016 = sqldf(query)

In [55]:
forest_area_decrease_btw_1990_2016

,region,forest_perc_1990,forest_perc_2016,forest_perc_decrease
0,Latin America & Caribbean,51.03,46.16,-4.87
1,Sub-Saharan Africa,30.67,28.79,-1.88
2,World,32.42,31.38,-1.04
3,Middle East & North Africa,1.78,2.07,0.29
4,North America,35.65,36.04,0.39
5,East Asia & Pacific,25.78,26.36,0.58
6,Europe & Central Asia,37.28,38.04,0.76
7,South Asia,16.51,17.51,1.00


In [54]:
# Which 5 countries saw the largest amount decrease in forest area from 1990 to 2016? 
# What was the difference in forest area for each?
query = """
WITH top_forest_area_decrease_btw_1990_2016 AS (SELECT a.country_name, a.region,  a.forest_area_sqkm -  b.forest_area_sqkm forest_area_loss

FROM (SELECT country_name, region, forest_area_sqkm 
    FROM forestation
    WHERE year = 1990) a
JOIN (SELECT country_name, region, forest_area_sqkm 
    FROM forestation
    WHERE year = 2016) b
ON a.country_name = b.country_name
ORDER BY 3 DESC
LIMIT 20)

SELECT *
FROM top_forest_area_decrease_btw_1990_2016 
WHERE forest_area_loss IS NOT NULL
LIMIT 6

"""
top_forest_area_decrease_btw_1990_2016 = sqldf(query)

# WITH top_forest_area_decrease_btw_1990_2016 AS (

# )
# SELECT *
# FROM top_forest_area_decrease_btw_1990_2016 
# WHERE forest_area_loss IS NOT NULL;


In [55]:
top_forest_area_decrease_btw_1990_2016

,country_name,region,forest_area_loss
0,World,World,1.324449e+06
1,Brazil,Latin America & Caribbean,5.415100e+05
2,Indonesia,East Asia & Pacific,2.821940e+05
3,Myanmar,East Asia & Pacific,1.072340e+05
4,Nigeria,Sub-Saharan Africa,1.065060e+05
5,Tanzania,Sub-Saharan Africa,1.023200e+05


#### COUNTRY-LEVEL DETAIL
Instructions on how to answer the questions

Answering these questions will help you add information to the template.
Use these questions as guides to write SQL queries.
Use the output from the query to answer these questions.
a. Which 5 countries saw the largest amount decrease in forest area from 1990 to 2016? What was the difference in forest area for each?

b. Which 5 countries saw the largest percent decrease in forest area from 1990 to 2016? What was the percent change to 2 decimal places for each?

c. If countries were grouped by percent forestation in quartiles, which group had the most countries in it in 2016?

d. List all of the countries that were in the 4th quartile (percent forest > 75%) in 2016.

e. How many countries had a percent forestation higher than the United States in 2016?

In [59]:
# Which 5 countries saw the largest percent decrease in forest area from 1990 to 2016? 
# What was the percent change to 2 decimal places for each?
query = """
WITH top_forest_area_largest_perc_increase_btw_1990_2016 AS (SELECT a.country_name, a.region, 
    (a.forest_area_sqkm-b.forest_area_sqkm)/a.forest_area_sqkm AS forest_area_perc_loss
    FROM (SELECT country_name, region, forest_area_sqkm 
    FROM forestation
    WHERE year = 1990) a
JOIN (SELECT country_name, region, forest_area_sqkm 
FROM forestation
WHERE year = 2016) b
ON a.country_name = b.country_name
ORDER BY 3 DESC
LIMIT 20)

SELECT *
FROM top_forest_area_largest_perc_increase_btw_1990_2016
WHERE forest_area_perc_loss IS NOT NULL
LIMIT 6
    
    """
top_forest_area_largest_perc_decrease_btw_1990_2016 = sqldf(query)

In [60]:
top_forest_area_largest_perc_decrease_btw_1990_2016

,country_name,region,forest_area_perc_loss
0,Togo,Sub-Saharan Africa,0.754453
1,Nigeria,Sub-Saharan Africa,0.617999
2,Uganda,Sub-Saharan Africa,0.591286
3,Mauritania,Sub-Saharan Africa,0.467470
4,Honduras,Latin America & Caribbean,0.450344
5,Pakistan,South Asia,0.434507


In [63]:
# Which 5 countries saw the largest percent increase in forest area from 1990 to 2016? 
# What was the percent change to 2 decimal places for each?
query = """

WITH top_forest_area_largest_perc_increase_btw_1990_2016 AS (SELECT a.country_name, a.region, 
    (a.forest_area_sqkm-b.forest_area_sqkm)/a.forest_area_sqkm AS forest_area_perc_loss
    FROM (SELECT country_name, region, forest_area_sqkm 
    FROM forestation
    WHERE year = 1990) a
JOIN (SELECT country_name, region, forest_area_sqkm 
FROM forestation
WHERE year = 2016) b
ON a.country_name = b.country_name
ORDER BY 3
LIMIT 20)

SELECT *
FROM top_forest_area_largest_perc_increase_btw_1990_2016
WHERE forest_area_perc_loss IS NOT NULL
LIMIT 6;

"""
top_forest_area_largest_perc_increase_btw_1990_2016 = sqldf(query)

In [64]:
top_forest_area_largest_perc_increase_btw_1990_2016

,country_name,region,forest_area_perc_loss
0,Iceland,Europe & Central Asia,-2.136646
1,French Polynesia,East Asia & Pacific,-1.818182
2,Bahrain,Middle East & North Africa,-1.772727
3,Uruguay,Latin America & Caribbean,-1.341113
4,Dominican Republic,Latin America & Caribbean,-0.824615
5,Kuwait,Middle East & North Africa,-0.811594


In [66]:
# If countries were grouped by percent forestation in quartiles, 
# which group had the most countries in it in 2016?
query = """

SELECT ntile, COUNT(*)
FROM (SELECT country_name, forest_perc, NTILE(4) OVER (PARTITION BY forest_perc)
FROM forestation
WHERE year = 2016 AND forest_perc IS NOT NULL) sub
GROUP BY 1;
"""
countries_ntile_forestation_2016 = sqldf(query)

In [216]:
# List all of the countries that were in the 4th quartile (percent forest > 75%) in 2016.
query = """

SELECT ntile, COUNT(*)
FROM (SELECT country_name, forest_perc, NTILE(4) OVER (PARTITION BY forest_perc)
FROM forestation
WHERE year = 2016 AND forest_perc IS NOT NULL) sub
GROUP BY 1;

"""
countries_4tile_forestation_2016 = sqldf(query)

In [218]:
countries_4tile_forestation_2016

In [ ]:
# How many countries had a percent forestation higher than the United States in 2016?
query = """
SELECT ntile, COUNT(*)
FROM (SELECT country_name, forest_perc, NTILE(4) OVER (ORDER BY forest_perc)
FROM forestation
WHERE year = 2016 AND forest_perc IS NOT NULL) sub
GROUP BY 1;
"""
countries_ntile_higher_than_US_2016 = sqldf(query)


In [ ]:
countries_ntile_higher_than_US_2016

In [77]:
query = """
WITH countries_with_highest_foerestation_loss AS(SELECT a.country_name, a.forest_area_sqkm forest_90, b.forest_area_sqkm forest_16, a.forest_area_sqkm-b.forest_area_sqkm forest_loss
FROM (SELECT country_name, forest_area_sqkm 
FROM forestation
WHERE year = 1990) a
JOIN (SELECT country_name, forest_area_sqkm 
FROM forestation
WHERE year = 2016) b
ON a.country_name = b.country_name
ORDER BY 4
LIMIT 20
)

SELECT *
FROM countries_with_highest_foerestation_loss
WHERE forest_loss IS NOT NULL

"""
countries_with_highest_foerestation_loss = sqldf(query)

In [78]:
countries_with_highest_foerestation_loss

,country_name,forest_90,forest_16,forest_loss
0,China,1.571406e+06,2.098635e+06,-527229.0620
1,United States,3.024500e+06,3.103700e+06,-79200.0000
2,India,6.393900e+05,7.086040e+05,-69213.9844
3,Russian Federation,8.089500e+06,8.148895e+06,-59395.0000
4,Vietnam,9.363000e+04,1.490200e+05,-55390.0000
5,Spain,1.380949e+05,1.845200e+05,-46425.0977


In [80]:
query = """

SELECT a.country_name, a.forest_perc forest_90, b.forest_perc forest_16, a.forest_perc-b.forest_perc forest_loss
FROM (SELECT country_name, forest_perc 
FROM forestation
WHERE year = 1990) a
JOIN (SELECT country_name, forest_perc 
FROM forestation
WHERE year = 2016) b
ON a.country_name = b.country_name
ORDER BY 4 
LIMIT 1;
"""
smaller_country_listed_at_top = sqldf(query)

In [81]:
smaller_country_listed_at_top

In [240]:
query = """

WITH forestation_quartiles_2016 AS (SELECT country_name, forest_perc,
CASE WHEN forest_perc > 0.75 THEN 4
     WHEN forest_perc <= 0.75 AND forest_perc > 0.5 THEN 3
     WHEN forest_perc <= 0.5 AND forest_perc > 0.25 THEN 2
     WHEN forest_perc <= 0.25 THEN 1
END AS level
FROM forestation
WHERE year = 2016)
SELECT level, COUNT(*)
FROM forestation_quartiles_2016
GROUP BY 1
"""
countries_grouped_by_forestation_quartiles_2016 = sqldf(query)

In [242]:
countries_grouped_by_forestation_quartiles_2016

,level,COUNT(*)
0,NaN,13
1,1.0,85
2,2.0,73
3,3.0,38
4,4.0,9


In [236]:
query = """
WITH quartile_countries_2016 AS (SELECT country_name, region, forest_perc,
CASE WHEN forest_perc > 0.75 THEN 4
     WHEN forest_perc <= 0.75 AND forest_perc > 0.5 THEN 3
     WHEN forest_perc <= 0.5 AND forest_perc > 0.25 THEN 2
     WHEN forest_perc <= 0.25 THEN 1
END AS level
FROM forestation
WHERE year = 2016)
SELECT country_name, region, forest_perc
FROM quartile_countries_2016
WHERE level = 4
"""
quartile_countries_2016 = sqldf(query)

In [237]:
quartile_countries_2016

,country_name,region,forest_perc
0,American Samoa,East Asia & Pacific,0.875001
1,"Micronesia, Fed. Sts.",East Asia & Pacific,0.918572
2,Gabon,Sub-Saharan Africa,0.900376
3,Guyana,Latin America & Caribbean,0.839014
4,Lao PDR,East Asia & Pacific,0.821082
5,Palau,East Asia & Pacific,0.876068
6,Solomon Islands,East Asia & Pacific,0.778635
7,Suriname,Latin America & Caribbean,0.982577
8,Seychelles,Sub-Saharan Africa,0.884111
